In [1]:
import time
from re import I
import wandb
import datetime
import pdb
from plotly import express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from logging import info, debug
from tkinter import W
import numpy as np
import os
import logging
import numpy as np
from tqdm import tqdm
import matplotlib
from PIL import Image
from skimage import io
import pprint
from models.utils import show_progress
import numpy as np
from logging import log, info, debug
from PIL import Image
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2 as cv
import sys
from models.models import build_resnet, build_siamese_autoencoder
from losses import DiceLoss
from metrics import DiceMetric
from matplotlib import pyplot as plt
from params import *
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

sys.path.append('/home/aadi/projects/pgrad-thesis/src/models')
sys.path.append('/home/aadi/projects/pgrad-thesis/data')
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

io.use_plugin('pil')
matplotlib.use('TkAgg')

pp = pprint.PrettyPrinter(indent=4)

2022-02-09 18:10:59,804 : Line: 117 - Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3060 Ti, compute capability 8.6
2022-02-09 18:10:59.801029: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-09 18:10:59.804072: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-09 18:10:59.804196: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-09 18:10:59.804356: I tensorflow/stream_executor/cuda/cuda_gpu_exe

1 Physical GPUs, 1 Logical GPUs


2022-02-09 18:10:59.805915: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-09 18:10:59.806042: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-09 18:10:59.806153: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-09 18:11:00.088372: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-09 18:11:00.088510: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [2]:
import build_data
from models.models import build_siamese_autoencoder
# help(build_data)

train, val = build_data.build_data_horizontal_separate(25, 100)
model = build_siamese_autoencoder()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
left (InputLayer)               [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 16) 208         left[0][0]                       
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 16) 1040        conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 128, 128, 16) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [3]:
# initialize METRICS for Tracking progress
train_dice = DiceMetric()
test_dice = DiceMetric()
val_dice = DiceMetric()

# initialize Dice LOSS for training step
dice_loss = DiceLoss()
bce_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
#--------------------------------------TENSORFLOW TRAINING LOOP STEPS---------------------------------#

@tf.function
def train_step(model, input, labels, optimizer):
    """Executes one training step and returns the loss.

    This function computes the loss and gradients, and uses the latter to
    update the model's parameters.
    """
    with tf.GradientTape() as tape:
        logits = model(input, training=True)
        # Instantiating DiceLoss() is for passing into model constructor
        print(logits.shape)
        # loss = dice_loss(labels, logits)
        loss = bce_loss(labels, logits)
    info(loss)

    grad = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grad, model.trainable_variables))

    train_dice.update_state(labels, logits)
    return loss

@tf.function
def val_step(model, input, labels):
    logits = model(input, training=False)
    val_dice.update_state(labels, logits)

    loss = bce_loss(labels, logits)
    return loss

@tf.function
def test_step(model, input, labels):
    test_logits = model(input, training=False)
    test_dice.update_state(labels, test_logits)

optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)

In [12]:
import sys
from cucim.skimage.transform import resize
from distutils.debug import DEBUG
import os
import logging
import numpy as np
from PIL import Image
import tensorflow as tf
from models.utils import show_progress
import cv2 as cv
import skimage
from tqdm import tqdm
from difference_functions import basic_subtract

logging.basicConfig(level=DEBUG)
TRAIN_DIR = 'data/train'
VAL_DIR = 'data/val'
TEST_DIR = 'data/test'


def _normalize(images, input_mask):
    before_image, after_image = images
    before_image = tf.cast(before_image, tf.float32) / 255.0
    after_image = tf.cast(after_image, tf.float32) / 255.0
    input_mask -= 1
    return (before_image, after_image), input_mask


def difference_data(data_path, difference_function=None):
    '''Assume three folders exist in data_path: time1, time2, label'''

    if not difference_function:
        difference_function = basic_subtract

    assert 'differenced' not in os.listdir(
        data_path), 'Differenced folder exists'
    assert set(os.listdir(data_path)) == set(
        ['time1', 'time2', 'label']), "Folders not present OR too many folders"

    before_path = os.path.join(data_path, 'time1')
    after_path = os.path.join(data_path, 'time2')
    label_path = os.path.join(data_path, 'label')

    assert len(os.listdir(before_path)) == len(os.listdir(after_path)
                                               ), "Length of before images different from length of after images"

    os.mkdir(os.path.join(data_path, 'differenced'))

    before_image_list = os.listdir(before_path)
    after_image_list = os.listdir(after_path)
    label_image_list = os.listdir(label_path)

    for i in tqdm(range(len(os.listdir(before_path)))):
        before_image_path = os.path.join(
            data_path, 'time1', before_image_list[i])
        after_image_path = os.path.join(
            data_path, 'time2', after_image_list[i])
        label_image_path = os.path.join(
            data_path, 'label', label_image_list[i])
        before_img = skimage.io.imread(before_image_path)
        after_img = skimage.io.imread(after_image_path)

        difference = difference_function(before_img, after_img)
        skimage.io.imsave(os.path.join(
            *[data_path, 'differenced', f'{i}.png']), difference)


def build_differenced_data(N=-1, batch_size=100, train=True, val=False, test=False):
    '''Assumes differenced images are in data/train/differenced, data/val/differenced, data/test/differenced and loads into single tf.data.Dataset with labels,
        returns tf.data.Dataset of (differenced, labels)
    '''
    autotune = tf.data.AUTOTUNE

    if train:
        main_dir = TRAIN_DIR
    elif val:
        main_dir = VAL_DIR
    else:
        main_dir = TEST_DIR

    diff_list = os.listdir(main_dir + '/differenced')[:N]
    label_list = os.listdir(main_dir + '/label')[:N]

    diff_list = [np.array(Image.open(main_dir + '/differenced/' + fname))
                 for fname in diff_list]
    label_list = [np.array(Image.open(main_dir + '/label/' + fname))
                  for fname in label_list]

    diff_list = [tf.convert_to_tensor(
        image, dtype=tf.float32) for image in diff_list]
    label_list = [tf.convert_to_tensor(
        image, dtype=tf.float32) for image in label_list]

    ds = tf.data.Dataset.from_tensor_slices((diff_list, label_list))

    del diff_list
    del label_list

    return ds.batch(batch_size).prefetch(buffer_size=autotune)

# Uses too much memory


def _build_train_arrays(N=-1):
    raise MemoryError(
        f'{_build_train_arrays.__name__} exceeds Protobuf memory usage')
    train_before_list = os.listdir(TRAIN_DIR + '/time1')[:N]
    train_after_list = os.listdir(TRAIN_DIR + '/time2')[:N]
    train_label_list = os.listdir(TRAIN_DIR + '/label')[:N]
    before_train = np.array([np.array(Image.open(TRAIN_DIR + '/time1/' + fname))
                            for fname in tqdm(train_before_list)])
    after_train = np.array([np.array(Image.open(TRAIN_DIR + '/time2/' + fname))
                            for fname in tqdm(train_after_list)])
    label_train = np.array([np.array(Image.open(TRAIN_DIR + '/label/' + fname))
                            for fname in tqdm(train_label_list)])

    return before_train, after_train, label_train


def _build_val_arrays(N=-1):
    raise MemoryError(
        f'{_build_val_arrays} exceeds Protobuf memory usage')
    val_before_list = os.listdir(VAL_DIR + '/time1')[:N]
    val_after_list = os.listdir(VAL_DIR + '/time2')[:N]
    val_label_list = os.listdir(VAL_DIR + '/label')[:N]
    before_val = np.array([np.array(Image.open(VAL_DIR + '/time1/' + fname))
                           for fname in show_progress(val_before_list)])
    after_val = np.array([np.array(Image.open(VAL_DIR + '/time2/' + fname))
                          for fname in show_progress(val_after_list)])
    label_val = np.array([np.array(Image.open(VAL_DIR + '/label/' + fname))
                          for fname in show_progress(val_label_list)])

    return before_val, after_val, label_val


def build_diff_data_rgb(batch_size, N=-1, image_shape=None):
    before_train, after_train, train_label = _build_train_arrays(N=N)
    before_val, after_val, val_label = _build_val_arrays(N=N)

    train_diff = np.array([basic_subtract(b, a)
                           for (b, a) in zip(before_train, after_train)])
    val_diff = np.array([basic_subtract(b, a)
                         for (b, a) in zip(before_val, after_val)])

    if image_shape:
        # reshape if pretrained weights expect different input sizes
        train_diff = [cv.resize(img, image_shape) for img in train_diff]
        val_diff = [cv.resize(img, image_shape) for img in val_diff]
        train_label = [cv.resize(img, image_shape) for img in train_label]
        val_label = [cv.resize(img, image_shape) for img in val_label]

    train_data = tf.data.Dataset.from_tensor_slices(
        (train_diff, train_label)).batch(batch_size=batch_size)
    val_data = tf.data.Dataset.from_tensor_slices(
        (val_diff, val_label)).batch(batch_size=batch_size)

    return train_data, val_data


def _normalize(img):
    img = tf.cast(img, tf.float32) / 255.0
    return img


def decode_grey(img):
    img = tf.io.decode_png(img, channels=1)
    return img


def decode_rgb(img):
    img = tf.io.decode_png(img, channels=3)
    return img


def process_path_rgb(fp):
    img = tf.io.read_file(fp)
    img = decode_rgb(img)
    img = _normalize(img)
    return img


def process_path_bin(fp: str) -> tf.Tensor:
    img = process_path_grey(fp)
    img = binarize_label(img)
    return img


def process_path_grey(fp):
    img = tf.io.read_file(fp)
    img = decode_grey(img)
    return img


def binarize_label(t: tf.Tensor) -> tf.Tensor:
    t = tf.where(t == 255, 1, 0)
    t = tf.cast(t, dtype=tf.uint8)
    return t


def build_data_horizontal_separate(batch_size, take, buffer_size=1000):

    def train_gen(split='train', data_path='../data/'):
        path = data_path + split
        for t1, t2, l in zip(sorted(os.listdir(path+'/time1')), sorted(os.listdir(path+'/time2')), sorted(os.listdir(path+'/label'))):
            # get full paths

            t1 = process_path_rgb(f'../data/{split}/time1/' + t1)
            t2 = process_path_rgb(f'../data/{split}/time2/' + t2)
            l = process_path_bin(f'../data/{split}/label/' + l)

            yield (t1, t2), l

    def val_gen(split='val', data_path='../data/'):
        path = data_path + split
        for t1, t2, l in zip(sorted(os.listdir(path+'/time1')), sorted(os.listdir(path+'/time2')), sorted(os.listdir(path+'/label'))):
            # get full paths

            t1 = process_path_rgb(f'../data/{split}/time1/' + t1)
            t2 = process_path_rgb(f'../data/{split}/time2/' + t2)
            l = process_path_bin(f'../data/{split}/label/' + l)

            yield (t1, t2), l

    train_ds = tf.data.Dataset.from_generator(
        train_gen, output_types=((tf.float32, tf.float32), tf.uint8))
    val_ds = tf.data.Dataset.from_generator(
        val_gen, output_types=((tf.float32, tf.float32), tf.uint8))

    train_batches = (
        train_ds
        .cache()
        .shuffle(buffer_size)
        .batch(batch_size)
        .take(take)
        .prefetch(buffer_size=tf.data.AUTOTUNE))

    val_batches = (
        val_ds
        .cache()
        .shuffle(buffer_size)
        .batch(batch_size)
        .take(take)
        .prefetch(buffer_size=tf.data.AUTOTUNE))

    return train_batches, val_batches


In [20]:
train, val = build_data_horizontal_separate(batch_size=25, take=100)
loss_fn = DiceLoss() # tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


for (x, y) in train.take(25):
    y_pred = model(x)
    print(loss_fn(y, y_pred))

tf.Tensor(1.5058184, shape=(), dtype=float32)
tf.Tensor(1.3656359, shape=(), dtype=float32)
tf.Tensor(1.6815155, shape=(), dtype=float32)
tf.Tensor(1.5150565, shape=(), dtype=float32)
tf.Tensor(1.5537194, shape=(), dtype=float32)
tf.Tensor(1.587725, shape=(), dtype=float32)
tf.Tensor(1.7112675, shape=(), dtype=float32)
tf.Tensor(1.6290858, shape=(), dtype=float32)
tf.Tensor(1.6060952, shape=(), dtype=float32)
tf.Tensor(1.5044984, shape=(), dtype=float32)
tf.Tensor(1.5482565, shape=(), dtype=float32)
tf.Tensor(1.6204467, shape=(), dtype=float32)
tf.Tensor(1.555047, shape=(), dtype=float32)
tf.Tensor(1.4933072, shape=(), dtype=float32)
tf.Tensor(1.587381, shape=(), dtype=float32)
tf.Tensor(1.5827873, shape=(), dtype=float32)
tf.Tensor(1.5489775, shape=(), dtype=float32)
tf.Tensor(1.5929946, shape=(), dtype=float32)
tf.Tensor(1.5676515, shape=(), dtype=float32)
tf.Tensor(1.7583501, shape=(), dtype=float32)
tf.Tensor(1.4065365, shape=(), dtype=float32)
tf.Tensor(1.543359, shape=(), dtype=f

2022-02-09 18:29:10.294541: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [ ]:
for x, y in train_data:
    print(f'Max-y: {tf.math.reduce_max(y)}, Min-y: {tf.math.reduce_min(y)}')

  Cloning https://github.com/tensorflow/examples.git to /tmp/pip-req-build-g8r77xmv
  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/examples.git /tmp/pip-req-build-g8r77xmv
  Resolved https://github.com/tensorflow/examples.git to commit 97dd38c31090290543e1f829fbbaddd192e8cc19
  Preparing metadata (setup.py) ... done
  Created wheel for tensorflow-examples: filename=tensorflow_examples-97dd38c31090290543e1f829fbbaddd192e8cc19_-py3-none-any.whl size=268468 sha256=9a1c084a83c96b881fd5f9837b937f3dde4fabcc20df03ea9a23dae2d2c67958
  Stored in directory: /tmp/pip-ephem-wheel-cache-uk95prwr/wheels/4e/f5/c2/bfe75b834c9028b2529023bf74541c46ead531b513a8010d21
Failed to build tensorflow-examples
  Running setup.py install for tensorflow-examples ... done
  DEPRECATION: tensorflow-examples was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. Di

ModuleNotFoundError: No module named 'tensorflow_datasets'